In [1]:
import timm
import torch
from torchsummary import summary

import sys
sys.path.append('..')
from dataset import ecg_dataset
from models import *

from teacher_model import TeacherModel

In [2]:
resnets = timm.list_models('resnet34*', pretrained=True)
print("ResNet34 可选:", resnets)

ResNet34 可选: ['resnet34.a1_in1k', 'resnet34.a2_in1k', 'resnet34.a3_in1k', 'resnet34.bt_in1k', 'resnet34.gluon_in1k', 'resnet34.ra4_e3600_r224_in1k', 'resnet34.tv_in1k', 'resnet34d.ra2_in1k']


In [26]:
# 查找 MobileNet V3 Small
mobs = timm.list_models('mobilenetv3_small*', pretrained=True)
print("MobileNetV3 Small 可选:", mobs)

MobileNetV3 Small 可选: ['mobilenetv3_small_050.lamb_in1k', 'mobilenetv3_small_075.lamb_in1k', 'mobilenetv3_small_100.lamb_in1k']


In [3]:
t_model = timm.create_model('resnet34.a1_in1k').to('cuda')
s_model = mobilenetv1().to('cuda')

In [5]:
x = torch.randn(1, 3, 224, 224).to('cuda')
emb = t_model.forward_features(x)
emb.shape

torch.Size([1, 512, 7, 7])

In [6]:
t_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [24]:
from functools import partial

s_features = {}
def hook(module, input, output, name): 
    s_features[name] = output.clone().detach().shape

for k, m in s_model.named_modules():
    m.register_forward_hook(partial(hook, name=k))
x = torch.randn(1, 3, 224, 224).to('cuda')  
y = s_model(x)
for k in s_features.keys():
    print(k, s_features[k]) 

model.0.0 torch.Size([1, 32, 112, 112])
model.0.1 torch.Size([1, 32, 112, 112])
model.0.2 torch.Size([1, 32, 112, 112])
model.0 torch.Size([1, 32, 112, 112])
model.1.0 torch.Size([1, 32, 112, 112])
model.1.1 torch.Size([1, 32, 112, 112])
model.1.2 torch.Size([1, 32, 112, 112])
model.1.3 torch.Size([1, 64, 112, 112])
model.1.4 torch.Size([1, 64, 112, 112])
model.1.5 torch.Size([1, 64, 112, 112])
model.1 torch.Size([1, 64, 112, 112])
model.2.0 torch.Size([1, 64, 56, 56])
model.2.1 torch.Size([1, 64, 56, 56])
model.2.2 torch.Size([1, 64, 56, 56])
model.2.3 torch.Size([1, 128, 56, 56])
model.2.4 torch.Size([1, 128, 56, 56])
model.2.5 torch.Size([1, 128, 56, 56])
model.2 torch.Size([1, 128, 56, 56])
model.3.0 torch.Size([1, 128, 56, 56])
model.3.1 torch.Size([1, 128, 56, 56])
model.3.2 torch.Size([1, 128, 56, 56])
model.3.3 torch.Size([1, 128, 56, 56])
model.3.4 torch.Size([1, 128, 56, 56])
model.3.5 torch.Size([1, 128, 56, 56])
model.3 torch.Size([1, 128, 56, 56])
model.4.0 torch.Size([1, 

In [25]:
from functools import partial

t_features = {}
def hook(module, input, output, name): 
    t_features[name] = output.clone().detach().shape

for k, m in t_model.named_modules():
    m.register_forward_hook(partial(hook, name=k))
x = torch.randn(1, 3, 224, 224).to('cuda')  
y = t_model(x)
for k in t_features.keys():
    print(k, t_features[k]) 

conv1 torch.Size([1, 64, 112, 112])
bn1 torch.Size([1, 64, 112, 112])
act1 torch.Size([1, 64, 112, 112])
maxpool torch.Size([1, 64, 56, 56])
layer1.0.conv1 torch.Size([1, 64, 56, 56])
layer1.0.bn1 torch.Size([1, 64, 56, 56])
layer1.0.drop_block torch.Size([1, 64, 56, 56])
layer1.0.act1 torch.Size([1, 64, 56, 56])
layer1.0.aa torch.Size([1, 64, 56, 56])
layer1.0.conv2 torch.Size([1, 64, 56, 56])
layer1.0.bn2 torch.Size([1, 64, 56, 56])
layer1.0.act2 torch.Size([1, 64, 56, 56])
layer1.0 torch.Size([1, 64, 56, 56])
layer1.1.conv1 torch.Size([1, 64, 56, 56])
layer1.1.bn1 torch.Size([1, 64, 56, 56])
layer1.1.drop_block torch.Size([1, 64, 56, 56])
layer1.1.act1 torch.Size([1, 64, 56, 56])
layer1.1.aa torch.Size([1, 64, 56, 56])
layer1.1.conv2 torch.Size([1, 64, 56, 56])
layer1.1.bn2 torch.Size([1, 64, 56, 56])
layer1.1.act2 torch.Size([1, 64, 56, 56])
layer1.1 torch.Size([1, 64, 56, 56])
layer1.2.conv1 torch.Size([1, 64, 56, 56])
layer1.2.bn1 torch.Size([1, 64, 56, 56])
layer1.2.drop_block t